Dependencies

In [1]:
import SimpleITK as sitk

Load Model

In [ ]:
#model = model from directory in repo 

Test MRI 

In [3]:
in_mha = sitk.ReadImage("out_inference/132_t1.mha")
print(in_mha)

Image (00000220929D6580)
  RTTI typeinfo:   class itk::Image<short,3>
  Reference Count: 1
  Modified Time: 1856
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 1826
  UpdateMTime: 1852
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [15, 320, 320]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [15, 320, 320]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [15, 320, 320]
  Spacing: [4.8, 1.0625, 1.0625]
  Origin: [-1495.12, -170, -129.26]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
4.8 0 0
0 1.0625 0
0 0 1.0625

  PointToIndexMatrix: 
0.208333 0 0
0 0.941176 0
0 0 0.941176

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (00000220921EE0E0)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int64,